### Loading the dataset

In [3]:
!pip uninstall jedi
from datasets import load_dataset
from pathlib import Path

In [4]:
# reading the jsonl dataset
data_dir = Path("../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl")

import pandas as pd    
jsonObj = pd.read_json(path_or_buf=data_dir, lines=True)



In [5]:
jsonObj.head()["answer"][2]

'reject'

In [6]:
jsonObj.columns

Index(['text', 'meta', '_input_hash', '_task_hash', 'label', 'score',
       'priority', 'spans', '_session_id', '_view_id', 'answer'],
      dtype='object')

### Huggingface dataset loading script
can be used later to publish the dataset on huggingface hub for better outreach

In [47]:
dataset = load_dataset("jsonl", data_files= ../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl)

SyntaxError: invalid syntax (<ipython-input-47-9df887cb1a05>, line 1)

### Testing the dataset loading script

In [26]:
from datasets import load_dataset


dataset = load_dataset("my_dataset_loading_script", name="condemnation", data_files="../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl")

Using custom data configuration condemnation-3bcd13f68545b6d6


in info


0 examples [00:00, ? examples/s]

filepath my_dataset_loading_script/ta_tweets.jsonl
{'text': 'Rose McGowan accuses <TARGET> of Rape - News Today - Breaking News https://t.co/q8AWAgxmwP via @YouTube', 'meta': {'tweet_id': '924121799004057600', 'body_target_mentions_target': 'weinstein', 'quoted_status_target_mentions': None, 'score': 0.5066827536}, '_input_hash': 1814849179, '_task_hash': -1213763831, 'label': 'condemnation', 'score': 0.5066827536, 'priority': 0.5066827536, 'spans': [], '_session_id': 'ta_tweets-test', '_view_id': 'classification', 'answer': 'reject'}
{'text': 'Over 300 Claims Against <TARGET>! https://t.co/6ztiuplOpl', 'meta': {'tweet_id': '924121911583526912', 'body_target_mentions_target': 'toback', 'quoted_status_target_mentions': None, 'score': 0.4642074704}, '_input_hash': 1276310490, '_task_hash': -302313038, 'label': 'condemnation', 'score': 0.4642074704, 'priority': 0.4642074704, 'spans': [], '_session_id': 'ta_tweets-test', '_view_id': 'classification', 'answer': 'reject'}
{'text': 'It’s horr

{'text': 'Four more women accuse <TARGET> of harassment, bringing total to at least a dozen https://t.co/7S5mmCBJGr https://t.co/LkfWrDcVb1', 'meta': {'tweet_id': '924121063482707968', 'body_target_mentions_target': 'halperin', 'quoted_status_target_mentions': None, 'score': 0.4488814175}, '_input_hash': -241144955, '_task_hash': 1040848974, 'label': 'condemnation', 'score': 0.4488814175, 'priority': 0.4488814175, 'spans': [], '_session_id': 'ta_tweets-test2', '_view_id': 'classification', 'answer': 'reject'}
{'text': '@TheFirstBladc » “I do love you. . I love you, <TARGET>. .”', 'meta': {'tweet_id': '924121113914875904', 'body_target_mentions_target': 'blaine', 'quoted_status_target_mentions': None, 'score': 0.4436924756}, '_input_hash': 164791188, '_task_hash': 489199799, 'label': 'condemnation', 'score': 0.4436924756, 'priority': 0.4436924756, 'spans': [], '_session_id': 'ta_tweets-test2', '_view_id': 'classification', 'answer': 'reject'}
{'text': '<TARGET>, Hefner and the Poor Excu

0 examples [00:00, ? examples/s]

filepath my_dataset_loading_script/ta_tweets.jsonl
{'text': 'Rose McGowan accuses <TARGET> of Rape - News Today - Breaking News https://t.co/q8AWAgxmwP via @YouTube', 'meta': {'tweet_id': '924121799004057600', 'body_target_mentions_target': 'weinstein', 'quoted_status_target_mentions': None, 'score': 0.5066827536}, '_input_hash': 1814849179, '_task_hash': -1213763831, 'label': 'condemnation', 'score': 0.5066827536, 'priority': 0.5066827536, 'spans': [], '_session_id': 'ta_tweets-test', '_view_id': 'classification', 'answer': 'reject'}
{'text': 'Over 300 Claims Against <TARGET>! https://t.co/6ztiuplOpl', 'meta': {'tweet_id': '924121911583526912', 'body_target_mentions_target': 'toback', 'quoted_status_target_mentions': None, 'score': 0.4642074704}, '_input_hash': 1276310490, '_task_hash': -302313038, 'label': 'condemnation', 'score': 0.4642074704, 'priority': 0.4642074704, 'spans': [], '_session_id': 'ta_tweets-test', '_view_id': 'classification', 'answer': 'reject'}
{'text': 'It’s horr

0 examples [00:00, ? examples/s]

filepath my_dataset_loading_script/ta_tweets.jsonl
{'text': 'Rose McGowan accuses <TARGET> of Rape - News Today - Breaking News https://t.co/q8AWAgxmwP via @YouTube', 'meta': {'tweet_id': '924121799004057600', 'body_target_mentions_target': 'weinstein', 'quoted_status_target_mentions': None, 'score': 0.5066827536}, '_input_hash': 1814849179, '_task_hash': -1213763831, 'label': 'condemnation', 'score': 0.5066827536, 'priority': 0.5066827536, 'spans': [], '_session_id': 'ta_tweets-test', '_view_id': 'classification', 'answer': 'reject'}
{'text': 'Over 300 Claims Against <TARGET>! https://t.co/6ztiuplOpl', 'meta': {'tweet_id': '924121911583526912', 'body_target_mentions_target': 'toback', 'quoted_status_target_mentions': None, 'score': 0.4642074704}, '_input_hash': 1276310490, '_task_hash': -302313038, 'label': 'condemnation', 'score': 0.4642074704, 'priority': 0.4642074704, 'spans': [], '_session_id': 'ta_tweets-test', '_view_id': 'classification', 'answer': 'reject'}
{'text': 'It’s horr

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'answer'],
        num_rows: 48
    })
    test: Dataset({
        features: ['text', 'answer'],
        num_rows: 48
    })
    validation: Dataset({
        features: ['text', 'answer'],
        num_rows: 48
    })
})